# Project 3
## Joshua Anderson

## General Aspects
1. There will be 10 periods. Players begin with 70 health.
2. Each period, the player harvests some amount of money (this amount is detailed below)
3. After harvesting, the player's health degenerates (this amount is detailed below)
4. After health degeneration, the player must spend money on Health Investments and Life Investments. Money spent on Health Investments increases health, while money spent on Life 5. 5. Investments gives the player Life Enjoyment. Any money not spent carries over into the next period.
6. A player dies if their health ever goes below 0. If a player dies, they receive 0 Life Enjoyment for the remaining periods.
7. The goal is to maximize total Life Enjoyment across all periods.

## Functions
 

Harvesting: 
1. The player earns income by harvesting black dots in a region designated by M x N cells. In our parameters (M = N = 100)
2. The player can select any set of contiguous W columns to harvest. In our parameters (W = 10)
3. When fully healthy, there are T black dots dispersed across M x W cells each period. Each black dot is worth v. In our parameters T = 100 and v = 1
4. The number of rows that can be harvested each period are given by: HarvestRows(H) = M $(1 - \gamma \frac{100 - H}{100})$ where $(\gamma = 1)$
5. The number of rows are reduced by disabling the rows in the upper and lower regions of the selected columns.
6. With these parameters, if health is 50 at the start of the period, the player has only 500 cells in which they can harvest black dots. With 80 health, they have 800 cells.
 

Degeneration:

Each period, the player loses (10 + CurrentPeriod) in health. i.e. 11 health the first period, 12 the second period, up to 20 in the last period.

Health Regeneration:

The equation for the amount of health regained given a certain Health Investment, I, and health after harvesting, H, is given by: 
$$ HealthRegained(I, H) = 100 \left( \frac{e^{k \cdot I}}{e^{k \cdot I} + \frac{100-H}{H}}\right)  - H $$

where (k = 0.01021).  Health cannot exceed 100, and is always rounded down to the nearest integer. 

Life Enjoyment:

The equation for the amount of Life Enjoyment given a certain Life Investment, L, is given by: 

$$ LifeEnjoyment(L, CurrentHealth) = c \left( \frac{CurrentHealth}{100} \right) \left( \frac{L}{L + \alpha} \right) $$

where (c = 464.53) and ($\alpha$ = 32).

CurrentHealth is the health the player has during the investment phase **INCLUDING** the amount regained this period through investments in health.

In [93]:
import numpy as np
import pandas as pd
import math
import random

# Constants

            player.health -= degenerate(player, i)
            I, L = player.get_investments()

            player.health += regenerate(player, I)

population_size = 10

starting_health = 70
min_health = 0

starting_enjoyment = 0

harvest_region_M = 100
harvest_region_N = 100
harvest_region_W = 10


In [82]:
# player class
class Player:
    def __init__(self, health, enjoyment):
        self.health = health
        self.enjoyment = enjoyment
        self.money = 0
    
    def select_column(self, N, W):
        self.col_index = random.randint(0, N - W)

    def get_dot_count(self):
        return sum(self.dots.sum(axis=1))
    
    def get_dots(self, W, disable_lower, disable_upper, data):
        start_idx = self.col_index
        end_idx = self.col_index + W
        self.dots = data.iloc[disable_lower:disable_upper,start_idx:end_idx]
        self.money += self.get_dot_count()

    def get_investments(self):
        I = random.randint(0, self.money)
        self.money -= I
        L = random.randint(0, self.money)
        self.money -= L
    
        return I, L
        


In [85]:
### functions ###

# Harvesting
# @params:
# M x N cells to a harvesting region
# W - contiguous columns to harvest
# T - black dots in harvesting region
# v - value of the black dots in a harvesting region
def harvest(players, M = 100, N = 100, W = 10, T = 100, v = 1, gamma = 1):
    region = pd.DataFrame()
    for i in range(N):
        value = T // W
        column = np.array([v]*value + [0]*(M-value))
        np.random.shuffle(column)
        region[i] = column

    for player in players:
        # harvest rows
        if player.health > min_health:
            reduced_gamma = gamma * ((100 - player.health) / 100)
            num_rows = M * (1 - reduced_gamma)
            disable_lower = int((M - num_rows) / 2)
            disable_upper = M - disable_lower
        
            player.select_column(N, W)
            player.get_dots(W, disable_lower, disable_upper, region)

# Degeneragtion
# @params:
# P - penalty added to the current period against health
def degenerate(player, curr_period, P = 10):
    return P + curr_period

# Health Regeneration
# @params:
# I - investment for regained health
# k - regen constant
# max_health - the value that health cannot exceed
def regenerate(player, I, k = 0.01021, max_health = 100):
    e_kI = math.exp(k*I)
    H_frac = (max_health - player.health) / player.health
    
    health_regen = math.floor(max_health * (e_kI / (e_kI + H_frac)) - player.health)

    if health_regen < 0:
        return 0
    elif health_regen > max_health:
        return max_health
    else:
        return health_regen

# Life Enjoyment - Fitness Function
# @params:
# L - infestment for live enjoyment
# c - constant multiplied by health
# alpha - constant that divides investment
def enjoyment(player, L, c = 464.53, alpha = 32):
    return c * (player.health / 100) * (L / (L + alpha))

# Generate Population
# @params:
# N - number of columns to choose from
# W - width of each column
# size - number of players in the population
def get_players(N, W, size):
    players = []
    for i in range(size):
        selection = random.randint(0, N - W)
        players.append(Player(starting_health, starting_enjoyment))
    return players

# Iterate Population

def tournament_survival(players, size):
    new_players = []
    for i in range(size):
        player_1 = players[random.randint(0, size-1)]
        player_2 = players[random.randint(0, size-1)]

        if player_1.enjoyment >= player_2.enjoyment:
            new_players.append(player_1)
        else:
            new_players.append(player_2)
    return new_players

In [97]:
players = get_players(harvest_region_N, harvest_region_W, population_size)
for i in range(periods):
    harvest(players)

    for player in players:
        player.health -= degenerate(player, i)

        if player.health > 0:
            I, L = player.get_investments()

            player.health += regenerate(player, I)
            player.enjoyment += enjoyment(player, L)
    
    players = tournament_survival(players, population_size)
    [print((player.health, player.enjoyment)) for player in players]
    print()


(61, 170.01798)
(70, 93.93828888888886)
(70, 93.93828888888886)
(72, 60.03156923076923)
(72, 60.03156923076923)
(70, 93.93828888888886)
(70, 93.93828888888886)
(65, 155.54716666666667)
(61, 170.01798)
(65, 155.54716666666667)

(65, 337.14026445642406)
(65, 337.14026445642406)
(60, 360.63400874811464)
(65, 337.14026445642406)
(65, 337.14026445642406)
(65, 337.14026445642406)
(56, 327.16728789743587)
(65, 337.14026445642406)
(56, 327.16728789743587)
(65, 337.14026445642406)

(32, 888.9120988868037)
(32, 888.9120988868037)
(32, 888.9120988868037)
(51, 505.0915087481146)
(32, 888.9120988868037)
(32, 888.9120988868037)
(32, 888.9120988868037)
(32, 888.9120988868037)
(32, 888.9120988868037)
(32, 888.9120988868037)

(-53, 1108.97561265373)
(-53, 1108.97561265373)
(-53, 1108.97561265373)
(-53, 1108.97561265373)
(-53, 1108.97561265373)
(-53, 1108.97561265373)
(-53, 1108.97561265373)
(-53, 1108.97561265373)
(-53, 1108.97561265373)
(-53, 1108.97561265373)

(-193, 1108.97561265373)
(-193, 1108.975